# Adler project

Work on EAGLE lens models: source reconstructions, synthetic imaging, comparison to true maps, ...


### Imports
`gleam` imports must happen from root of the directory, i.e. `/Users/phdenzel/adler/`.
This is why `..` is appended to the path.


In [ ]:
import sys
sys.path.append('..')
import os
import re
import numpy as np
from scipy import interpolate
from astropy.io import fits
import matplotlib.pyplot as plt

from gleam.multilens import MultiLens
from gleam.reconsrc import ReconSrc
from gleam.glass_interface import glass_renv, filter_env, export_state
glass = glass_renv()

%load_ext skip_kernel_extension


### Reading data
Reading in all relevant files as dictionaries with ids as keys and list of filenames as values


In [ ]:
rdir = "/Users/phdenzel/adler"
jsondir = rdir+"/json/"
statedir = rdir+"/states/v2/"
kappadir = rdir+"/kappa/"
keys = ["H1S0A0B90G0", "H1S1A0B90G0", "H2S1A0B90G0", "H2S2A0B90G0", "H2S7A0B90G0",
        "H3S0A0B90G0", "H3S1A0B90G0", "H4S3A0B0G90", "H10S0A0B90G0", "H13S0A0B90G0",
        "H23S0A0B90G0", "H30S0A0B90G0", "H36S0A0B90G0", "H160S0A90B0G0",
        "H234S0A0B90G0"]


In [ ]:
def an_sorted(data):
    """
    Perform an alpha-numeric, natural sort

    Args:
        data <list> - list of strings

    Kwargs:
        None

    Return:
        sorted <list> - the alpha-numerically, naturally sorted list of strings
    """
    def convert(text):
        return int(text) if text.isdigit() else text.lower()

    def an_key(key):
        return [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=an_key)


ls_jsons = an_sorted([os.path.join(jsondir, f) for f in os.listdir(jsondir)
                      if f.endswith('.json')])
ls_states = an_sorted([os.path.join(statedir, f) for f in os.listdir(statedir)
                       if f.endswith('.state')])
ls_kappas = an_sorted([os.path.join(kappadir, f) for f in os.listdir(kappadir)
                       if f.endswith('.kappa.fits')])


In [ ]:
jsons = {k: [f for f in ls_jsons if k in f] for k in keys}
filtered_states = {k: [f for f in ls_states
                       if k in f and f.endswith('_filtered.state')] for k in keys}
ls_states = [f for f in ls_states if not f.endswith('_filtered.state')]

prefiltered_fsynth10_states = {k: [f for f in ls_states
                                   if k in f and f.endswith('_filtered_synthf10.state')] for k in keys}
prefiltered_fsynth25_states = {k: [f for f in ls_states
                                   if k in f and f.endswith('_filtered_synthf25.state')] for k in keys}
prefiltered_fsynth50_states = {k: [f for f in ls_states
                                   if k in f and f.endswith('_filtered_synthf50.state')] for k in keys}
ls_states = [f for f in ls_states if not (f.endswith('_filtered_synthf10.state')
                                          or f.endswith('_filtered_synthf10.state')
                                          or f.endswith('_filtered_synthf10.state'))]

synthf10_states = {k: [f for f in ls_states
                       if k in f and f.endswith('_synthf10.state')] for k in keys}
synthf25_states = {k: [f for f in ls_states
                       if k in f and f.endswith('_synthf25.state')] for k in keys}
synthf50_states = {k: [f for f in ls_states
                       if k in f and f.endswith('_synthf50.state')] for k in keys}
ls_states = [f for f in ls_states if not (f.endswith('_synthf10.state')
                                          or f.endswith('_synthf25.state')
                                          or f.endswith('_synthf50.state'))]

states = {k: [f for f in ls_states if k in f] for k in keys}

kappa_map_files = {k: [f for f in ls_kappas if k in f] for k in keys}


### Inspection

The ensemble of a single state file is used

In [ ]:
# select a single file
print("\n# Selected file")
key = keys[0]
idx = 0
json = jsons[key][0]
state = filtered_states[key][idx]
state = states[key][idx]
print(json)
print(state)

In [ ]:
# gleamobject
print("\n# gleamobject")
with open(json) as f:
    ml = MultiLens.from_json(f)
print(ml.__v__)

In [ ]:
# recon_src
print("\n# recon_src")
args = (ml, state)
kwargs = dict(M=20, verbose=1)
recon_src = ReconSrc(*args, **kwargs)
#recon_src.chmdl()  # move to a single ensemble model

In [ ]:
# inverse projection matrix
print("\n# inverse projection matrix")
Mij_p = recon_src.inv_proj_matrix()
print(type(Mij_p))
print(Mij_p.shape)

In [ ]:
# (inverse of the inverse) projection matrix
print("\n# projection matrix (TODO: not an actual inverse yet)")
Mp_ij = recon_src.proj_matrix()
print(type(Mp_ij))
print(Mp_ij.shape)

In [ ]:
# image plane data arrays
print("\n# image plane data arrays")
data = recon_src.d_ij()  # 1d lens plane data
print(type(data))
print(data.shape)
lmap = recon_src.lens_map()  # 2d lens plane data
print(type(lmap))
print(lmap.shape)
print("\n# source plane data arrays")
# source plane data arrays
rsrc = recon_src.d_p()  # 1d source plane data
print(type(rsrc))
print(rsrc.shape)
rsrc_map = recon_src.plane_map()  # 2d source plane data
print(type(rsrc_map))
print(rsrc_map.shape)

# synthetic image
print("\n# synthetic image")
reproj = recon_src.reproj_map()
print(type(reproj))
print(reproj.shape)

#### Actual data plot

In [ ]:
data = recon_src.d_ij(flat=False)
kwargs = dict(vmax=data.max(), vmin=data.min())
plt.imshow(data, **kwargs)
plt.colorbar()
plt.axis('off')
plt.show()


#### Reconstructed source plot

In [ ]:
plt.imshow(recon_src.plane_map(), **kwargs)
plt.colorbar()
plt.axis('off')
plt.show()


#### Synthetic image plot

In [ ]:
plt.imshow(recon_src.reproj_map(), **kwargs)
plt.colorbar()
plt.axis('off')
plt.show()


#### Masked data plot

In [ ]:
plt.imshow(recon_src.lens_map(mask=True), **kwargs)
plt.colorbar()
plt.axis('off')
plt.show()

#### Residual map plot

In [ ]:
plt.imshow(recon_src.residual_map())  # data - synth
plt.colorbar()
plt.axis('off')
plt.show()

#### Residual statistics

In [ ]:
%%skip True
print("Sum of squared residuals (ensemble avg)")
resid = recon_src.reproj_residual()
print(resid)

print("Sum of squared residuals (for the first three ensemble models)")
residuals = []
N = len(recon_src.gls.models)
for i in range(N):
    recon_src.chmdl(i)
    r = recon_src.reproj_residual()
    residuals.append(r)
print(residuals[:3])

print("0th, 10th, 25th, and 50th percentile values")
rhi10 = np.percentile(residuals, 10, interpolation='higher')
rhi25 = np.percentile(residuals, 25, interpolation='higher')
rhi50 = np.percentile(residuals, 50, interpolation='higher')
rlo = 0
print(rlo, rhi10, rhi25, rhi50)

#### Histogram of an ensemble's residual distribution
    

In [ ]:
%%skip True

plt.hist(residuals, bins=35)
plt.axvline(rhi10)
plt.axvline(rhi25)
plt.axvline(rhi50)
plt.show()


#### Filtering and exporting the single state

In [ ]:
%%skip True

def synth_filter(args, percentiles=[10, 25, 50]):
    pass

In [ ]:
%%skip True

select10 = [i for i, r in enumerate(residuals) if rhi10 > r > rlo]
select25 = [i for i, r in enumerate(residuals) if rhi25 > r > rlo]
select50 = [i for i, r in enumerate(residuals) if rhi50 > r > rlo]
print("Number of selected models in 10th, 25th and 50th percentile")
print(len(select10))
print(len(select25))
print(len(select50))

In [ ]:
%%skip True

dirname = os.path.dirname(state)
basename = ".".join(os.path.basename(state).split('.')[:-1])
save10 = dirname + '/' + basename + '_synthf10.state'
save25 = dirname + '/' + basename + '_synthf25.state'
save50 = dirname + '/' + basename + '_synthf50.state'
print("Names of filtered states...")
print(save10)
print(save25)
print(save50)


In [ ]:
%%skip True

filtered_10 = filter_env(recon_src.gls, select10)
filtered_25 = filter_env(recon_src.gls, select25)
filtered_50 = filter_env(recon_src.gls, select50)
export_state(filtered_25, name=save25)
export_state(filtered_50, name=save50)


### Filtering ALL (previously unfiltered) states

In [ ]:
%%skip True

# states loop
for k in keys:
    json = jsons[k][0]
    with open(json) as f:
        ml = MultiLens.from_json(f)
    for state in states[k]:
        print(json, state)

        args = (ml, state)
        kwargs = dict(M=20, verbose=1)
        recon_src = ReconSrc(*args, **kwargs)

        residuals = []
        for i in range(len(recon_src.gls.models)):
            recon_src.chmdl(i)
            r = recon_src.reproj_residual()
            residuals.append(r)
        print("\nNumber of residual models:")
        print(len(residuals))

        rhi10 = np.percentile(residuals, 10, interpolation='higher')
        rhi25 = np.percentile(residuals, 25, interpolation='higher')
        rhi50 = np.percentile(residuals, 50, interpolation='higher')
        rlo = 0
        print("\n0th, 10th, 25th, and 50th percentile values")
        print(rlo, rhi10, rhi25, rhi50)

        select10 = [i for i, r in enumerate(residuals) if rhi10 > r > rlo]
        select25 = [i for i, r in enumerate(residuals) if rhi25 > r > rlo]
        select50 = [i for i, r in enumerate(residuals) if rhi50 > r > rlo]

        print("\nNumber of selected models")
        print(len(select10))
        print(len(select25))
        print(len(select50))

        dirname = os.path.dirname(state)
        basename = ".".join(os.path.basename(state).split('.')[:-1])
        save10 = dirname + '/' + basename + '_synthf10.state'
        save25 = dirname + '/' + basename + '_synthf25.state'
        save50 = dirname + '/' + basename + '_synthf50.state'

        filtered_10 = filter_env(recon_src.gls, select10)
        filtered_25 = filter_env(recon_src.gls, select25)
        filtered_50 = filter_env(recon_src.gls, select50)

        export_state(filtered_10, name=save10)
        export_state(filtered_25, name=save25)
        export_state(filtered_50, name=save50)


### Filtering ALL (previously filtered) states

In [ ]:
%%skip True

# filtered_states loop
for k in keys:
    json = jsons[k][0]
    with open(json) as f:
        ml = MultiLens.from_json(f)
    for state in filtered_states[k]:
        print(json, state)

        args = (ml, state)
        kwargs = dict(M=20, verbose=1)
        recon_src = ReconSrc(*args, **kwargs)

        residuals = []
        for i in range(len(recon_src.gls.models)):
            recon_src.chmdl(i)
            r = recon_src.reproj_residual()
            residuals.append(r)
        print("\nNumber of residual models")
        print(len(residuals))

        rhi10 = np.percentile(residuals, 10, interpolation='higher')
        rhi25 = np.percentile(residuals, 25, interpolation='higher')
        rhi50 = np.percentile(residuals, 50, interpolation='higher')
        rlo = 0
        print("\n0th, 10th, 25th, and 50th percentile values")
        print(rlo, rhi10, rhi25, rhi50)

        select10 = [i for i, r in enumerate(residuals) if rhi10 > r > rlo]
        select25 = [i for i, r in enumerate(residuals) if rhi25 > r > rlo]
        select50 = [i for i, r in enumerate(residuals) if rhi50 > r > rlo]

        print("\nNumber of subselected models")
        print(len(select10))
        print(len(select25))
        print(len(select50))

        dirname = os.path.dirname(state)
        basename = ".".join(os.path.basename(state).split('.')[:-1])
        save10 = dirname + '/' + basename + '_synthf10.state'
        save25 = dirname + '/' + basename + '_synthf25.state'
        save50 = dirname + '/' + basename + '_synthf50.state'

        filtered_10 = filter_env(recon_src.gls, select10)
        filtered_25 = filter_env(recon_src.gls, select25)
        filtered_50 = filter_env(recon_src.gls, select50)

        export_state(filtered_10, name=save10)
        export_state(filtered_25, name=save25)
        export_state(filtered_50, name=save50)


## Comparison to true EAGLE models

#### Reading the files

In [ ]:
%%skip True
kappa_map_files = {
    "H1S0A0B90G0":   [rdir+'/kappa/H1S0A0B90G0gridmap.kappa.fits'],
    "H1S1A0B90G0":   [rdir+'/kappa/H1S1A0B90G0gridmap.kappa.fits'],
    "H2S1A0B90G0":   [rdir+'/kappa/H2S1A0B90G0gridmap.kappa.fits'],
    "H2S2A0B90G0":   [rdir+'/kappa/H2S2A0B90G0gridmap.kappa.fits'],
    "H2S7A0B90G0":   [rdir+'/kappa/H2S7A0B90G0gridmap.kappa.fits'],
    "H3S0A0B90G0":   [rdir+'/kappa/H3S0A0B90G0gridmap.kappa.fits'],
    "H3S1A0B90G0":   [rdir+'/kappa/H3S1A0B90G0gridmap.kappa.fits'],
    "H4S3A0B0G90":   [rdir+'/kappa/H4S3A0B0G90gridmap.kappa.fits'],
    "H10S0A0B90G0":  [rdir+'/kappa/H10S0A0B90G0gridmap.kappa.fits'],
    "H13S0A0B90G0":  [rdir+'/kappa/H13S0A0B90G0gridmap.kappa.fits'],
    "H23S0A0B90G0":  [rdir+'/kappa/H23S0A0B90G0gridmap.kappa.fits'],
    "H30S0A0B90G0":  [rdir+'/kappa/H30S0A0B90G0gridmap.kappa.fits'],
    "H36S0A0B90G0":  [rdir+'/kappa/H36S0A0B90G0gridmap.kappa.fits'],
    "H160S0A90B0G0": [rdir+'/kappa/H160S0A90B0G0gridmap.kappa.fits'],
    "H234S0A0B90G0": [rdir+'/kappa/H234S0A0B90G0gridmap.kappa.fits']
}


In [ ]:
# EAGLE models
filepath = kappa_map_files["H3S0A0B90G0"][0]
# filepath = kappa_map_files["H10S0A0B90G0"][0]
# filepath = kappa_map_files["H36S0A0B90G0"][0]
eagle_kappa_map, eagle_hdrobj = fits.getdata(filepath, header=True)

# GLASS models
#statefile = states["H3S0A0B90G0"][0]
statefile = prefiltered_fsynth50_states["H3S0A0B90G0"][0]  # prefiltered_fsynth50_states["H3S0A0B90G0"][0]
# statefile = prefiltered_fsynth50_states["H10S0A0B90G0"][0]
# statefile = prefiltered_fsynth50_states["H36S0A0B90G0"][0]
glass_state = glass.glcmds.loadstate(statefile)
glass_state.make_ensemble_average()

kkwargs = dict(vmin=0, vmax=eagle_kappa_map.max())


#### The true EAGLE models

In [ ]:
eagle_pixrad = tuple(r//2 for r in eagle_kappa_map.shape)
eagle_maprad = eagle_pixrad[1]*eagle_hdrobj['CDELT2']*3600
eagle_mapextent = ((eagle_pixrad[0]+0.5)*eagle_hdrobj['CDELT1']*3600,
                  (eagle_pixrad[1]+0.5)*eagle_hdrobj['CDELT2']*3600)
eagle_cell_size = -3600*eagle_hdrobj['CDELT1']
assert(eagle_hdrobj['CUNIT1' == 'deg'])

print("Kappa map: {}".format(eagle_kappa_map.shape))
print("Pixrad {}".format(eagle_pixrad))
print("Maprad {}".format(eagle_maprad))
print("Mapextent {}".format(eagle_mapextent))
print("Cellsize {}".format(eagle_cell_size))


In [ ]:
plt.imshow(np.log10(eagle_kappa_map))
plt.colorbar()
plt.gca().axis('off')
plt.show()


#### Resample EAGLE model to match GLASS model

In [ ]:
def resample_eaglemodel(eagle_model, extent, shape, verbose=False):
    """
    Resample (usually downsample) an EAGLE model's kappa grid to match the specified scales and dimensions

    Args:
        eagle_model <tuple(np.ndarray, header object)> - EAGLE model with (data, hdr)
        extent <tuple/list> - extent of the output
        shape <tuple/list> - shape of the output

    Kwargs:
        verbose <bool> - verbose mode; print command line statements

    Return:
        kappa_resmap <np.ndarray> - resampled kappa grid
    """
    kappa_map = eagle_model[0]
    hdr = eagle_model[1]

    pixrad = tuple(r//2 for r in kappa_map.shape)
    maprad = pixrad[1]*hdr['CDELT2']*3600

    xmdl = np.linspace(-maprad, maprad, kappa_map.shape[0])
    ymdl = np.linspace(-maprad, maprad, kappa_map.shape[1])
    newx = np.linspace(extent[0], extent[1], shape[0])
    newy = np.linspace(extent[2], extent[3], shape[1])

    rescale = interpolate.interp2d(xmdl, ymdl, kappa_map)
    kappa_resmap = rescale(newx, newy)
    kappa_resmap[kappa_resmap < 0] = 0

    return kappa_resmap

In [ ]:
glass_objdta = glass_state.models[0]['obj,data'][0]
glsmdl_maprad = glass_objdta[0].basis.top_level_cell_size * (glass_objdta[0].basis.pixrad)
shape = glass_objdta[0].basis._to_grid(glass_objdta[1]['kappa'], 1).shape
extent = (-glsmdl_maprad, glsmdl_maprad, -glsmdl_maprad, glsmdl_maprad)
eagle_kappa_resmap = resample_eaglemodel(fits.getdata(filepath, header=True), extent, shape)

plt.imshow(np.log10(eagle_kappa_resmap))
plt.colorbar()
plt.gca().axis('off')
plt.show()


#### Residuals (EAGLE2GLASS)

In [ ]:
%%skip True

glass_objdta = glass_state.models[0]['obj,data'][0]
glass_kappa_map = glass_objdta[0].basis._to_grid(glass_objdta[1]['kappa'], 1)

resid_map = eagle_kappa_resmap - glass_kappa_map
resid2_map = resid_map * resid_map
r = np.sum(resid2_map)
print(r)


In [ ]:
%%skip True

plt.imshow(resid_map)
plt.colorbar()
plt.gca().axis('off')
plt.show()


#### Comparison loop for a single model (EAGLE2GLASS)

In [ ]:
N_models = len(glass_state.models)
print("N models {}".format(N_models))

kappa_resids = []
eagle_model = fits.getdata(filepath, header=True)
for m in glass_state.models:
    objdta = m['obj,data'][0]
    glass_kappa_map = objdta[0].basis._to_grid(objdta[1]['kappa'], 1)
    maprad = objdta[0].basis.top_level_cell_size * (objdta[0].basis.pixrad)
    shape = glass_kappa_map.shape
    extent = (-maprad, maprad, -maprad, maprad)

    kappa_resmap = resample_eaglemodel(eagle_model, extent, shape)

    # residuals
    resid_map = kappa_resmap - glass_kappa_map
    resid2_map = resid_map * resid_map
    r = np.sum(resid2_map)
    kappa_resids.append(r)

minidx = np.argmin(kappa_resids)
print("Best fitting model index: {}".format((minidx, kappa_resids[minidx])))
print(kappa_resids)


In [ ]:
objdta = glass_state.models[minidx]['obj,data'][0]
min_e2gkappa_map = objdta[0].basis._to_grid(objdta[1]['kappa'], 1)
maprad = objdta[0].basis.top_level_cell_size * (objdta[0].basis.pixrad)
shape = min_e2gkappa_map.shape
extent = (-maprad, maprad, -maprad, maprad)

kappa_resmap = resample_eaglemodel(eagle_model, extent, shape)

resid_map = kappa_resmap - min_e2gkappa_map
resid2_map = resid_map * resid_map

plt.imshow(min_e2gkappa_map, **kkwargs)
plt.colorbar()
plt.gca().axis('off')
plt.show()

plt.imshow(resid_map)
plt.colorbar()
plt.gca().axis('off')
plt.show()

#### The GLASS models

In [ ]:
glass_model = glass_state.models[0]
glassobj, glassdta = glass_model['obj,data'][0]
glass_kappa_map = glassobj.basis._to_grid(glassdta['kappa'], 1)

glass_pixrad = glassobj.basis.pixrad
glass_maprad = glassobj.basis.top_level_cell_size * (glassobj.basis.pixrad)
glass_mapextent = (-glassobj.basis.top_level_cell_size * (glassobj.basis.pixrad+0.5),
                 glassobj.basis.top_level_cell_size * (glassobj.basis.pixrad+0.5))
glass_cell_size = glassobj.basis.top_level_cell_size

print("Kappa map: {}".format(glass_kappa_map.shape))
print("Pixrad {}".format(glass_pixrad))
print("Maprad {}".format(glass_maprad))
print("Mapextent {}".format(glass_mapextent))
print("Cellsize {}".format(glass_cell_size))


In [ ]:
# kkwargs = dict()
plt.imshow(glass_kappa_map, **kkwargs)
plt.colorbar()
plt.gca().axis('off')
plt.show()


In [ ]:
glass_state.img_plot(obj_index=0, color='#fe4365')
glass_state.arrival_plot(glass_state.ensemble_average, obj_index=0, only_contours=True,
                         clevels=75, colors=['#603dd0'])
glass_state.arrival_plot(glass_state.ensemble_average, obj_index=1, only_contours=True,
                         clevels=50, colors=['#da9605'])
plt.show()

#### Resample GLASS model to match EAGLE model

In [ ]:
def resample_glassmodel(gls_model, extent, shape, verbose=False):
    """
    Resample (usually upsample) a GLASS model's kappa grid to match the specified scales and dimensions

    Args:
        gls_model <glass.LensModel object> - GLASS ensemble model
        extent <tuple/list> - extent of the output
        shape <tuple/list> - shape of the output

    Kwargs:
        verbose <bool> - verbose mode; print command line statements

    Return:
        kappa_resmap <np.ndarray> - resampled kappa grid
    """
    obj, data = gls_model['obj,data'][0]
    kappa_map = obj.basis._to_grid(data['kappa'], 1)
    pixrad = obj.basis.pixrad
    maprad = obj.basis.top_level_cell_size * (obj.basis.pixrad)
    mapextent = (-obj.basis.top_level_cell_size * (obj.basis.pixrad+0.5),
                 obj.basis.top_level_cell_size * (obj.basis.pixrad+0.5))
    cell_size = obj.basis.top_level_cell_size

    if verbose:
        print(obj)
        print("Kappa map: {}".format(kappa_map.shape))
        print("Pixrad {}".format(pixrad))
        print("Maprad {}".format(maprad))
        print("Mapextent {}".format(mapextent))
        print("Cellsize {}".format(cell_size))

    xmdl = np.linspace(-maprad, maprad, kappa_map.shape[0])
    ymdl = np.linspace(-maprad, maprad, kappa_map.shape[1])
    Xmdl, Ymdl = np.meshgrid(xmdl, ymdl)
    xnew = np.linspace(extent[0], extent[1], shape[0])
    ynew = np.linspace(extent[2], extent[3], shape[1])
    Xnew, Ynew = np.meshgrid(xnew, ynew)

    rescale = interpolate.Rbf(Xmdl, Ymdl, kappa_map)
    kappa_resmap = rescale(Xnew, Ynew)
    kappa_resmap[kappa_resmap < 0] = 0

    return kappa_resmap


In [ ]:
eagle_pixrad = tuple(r//2 for r in eagle_kappa_map.shape)
eagle_maprad = eagle_pixrad[1]*eagle_hdrobj['CDELT2']*3600
shape = eagle_kappa_map.shape
extent = [-eagle_maprad, eagle_maprad, -eagle_maprad, eagle_maprad]
glass_kappa_resmap = resample_glassmodel(glass_state.models[0], extent, shape)


In [ ]:
%%skip True

x = np.linspace(-mdl_maprad, mdl_maprad, mdl_kappa_map.shape[0])
y = np.linspace(-mdl_maprad, mdl_maprad, mdl_kappa_map.shape[1])
newx = np.linspace(-eagle_maprad, eagle_maprad, eagle_kappa_map.shape[0])
newy = np.linspace(-eagle_maprad, eagle_maprad, eagle_kappa_map.shape[1])

rescale = interpolate.interp2d(x, y, mdl_kappa_map, kind='linear')
# rescale = interpolate.RectBivariateSpline(x, y, mdl_kappa_map)
glass_kappa_resmap = rescale(newx, newy)


In [ ]:
plt.imshow(glass_kappa_resmap, **kkwargs)
plt.colorbar()
plt.gca().axis('off')
plt.show()


#### Residuals (GLASS2EAGLE)

In [ ]:
%%skip True

resid_map = eagle_kappa_map - glass_kappa_resmap
resid2_map = resid_map * resid_map
r = np.sum(resid2_map)
print(r)


In [ ]:
%%skip True

plt.imshow(resid_map)
plt.colorbar()
plt.gca().axis('off')
plt.show()


#### Comparison loop for a single model (GLASS2EAGLE)

In [ ]:
N_models = len(glass_state.models)
print("N models {}".format(N_models))

kappa_resids = []
extent = [-eagle_maprad, eagle_maprad, -eagle_maprad, eagle_maprad]
for m in glass_state.models:
    mdl_kappa_resmap = resample_glassmodel(m, extent, eagle_kappa_map.shape)

    # residuals
    resid_map = eagle_kappa_map - mdl_kappa_resmap
    resid2_map = resid_map * resid_map
    r = np.sum(resid2_map)
    kappa_resids.append(r)

minidx = np.argmin(kappa_resids)
print("Best fitting model index: {}".format((minidx, kappa_resids[minidx])))
print(kappa_resids)


In [ ]:
min_g2ekappa_map = resample_glassmodel(glass_state.models[minidx], extent, eagle_kappa_map.shape)
resid_map = eagle_kappa_map - min_g2ekappa_map
resid2_map = np.sqrt(resid_map * resid_map)

plt.imshow(min_g2ekappa_map, **kkwargs)
plt.colorbar()
plt.gca().axis('off')
plt.show()

plt.imshow(resid_map)
plt.colorbar()
plt.gca().axis('off')
plt.show()


## Check COM

The center of mass is defined as
$
\begin{equation}
\mathbf{R} = \frac{1}{M} \sum_{i=1}^{n}m_{i}\mathbf{r}_{i}
\end{equation}
$

In [ ]:
def center_of_mass(kappa_map, pixel_scale=1):
    """
    Calculate the 2D position of the center of mass

    Args:
        kappa_map <np.ndarray> - a 2D grid of kappa tiles

    Kwargs:
        pixel_scale <float> - the pixel scale

    Return:
        com <np.ndarray> - center of mass on the kappa coordinate grid as (x, y)
    """
    invM = 1/np.sum(kappa_map)
    r = np.array(np.where(kappa_map+2*np.abs(kappa_map).max())).T
    kappa = np.array([kappa_map.flatten()]*2).T
    return np.sum(invM*r*kappa*pixel_scale, axis=0)[::-1]

eagle_com = center_of_mass(eagle_kappa_map)
eagle_res_com = center_of_mass(eagle_kappa_resmap)
glassg2e_com = center_of_mass(min_g2ekappa_map)
glasse2g_com = center_of_mass(min_e2gkappa_map)

print("Center of mass for EAGLE model {}: {}".format(eagle_kappa_map.shape, eagle_com))
print("Center of mass for GLASS model {}: {}".format(min_g2ekappa_map.shape, glassg2e_com))

print("Center of mass for EAGLE model {}: {}".format(eagle_kappa_resmap.shape, eagle_res_com))
print("Center of mass for GLASS model {}: {}".format(min_e2gkappa_map.shape, glasse2g_com))

# assert all(np.round(eagle_com) == np.round([0.5*s for s in eagle_kappa_map.shape]))
# assert all(np.round(eagle_res_com) == np.round([0.5*s for s in eagle_kappa_resmap.shape]))
# assert all(np.round(glassg2e_com) == np.round([0.5*s for s in min_g2ekappa_map.shape]))
# assert all(np.round(glasse2g_com) == np.round([0.5*s for s in min_e2gkappa_map.shape]))


## Inertial tensor

The inertial tensor on a 2D grid is defined as
$
\begin{equation}
I = \left[ {\begin{array}{cc}
            \int \kappa x^2 & \int \kappa xy\\
            \int \kappa yx  & \int \kappa y^2\\
            \end{array}}
    \right]
\end{equation}
$
where $x$, $y$ are the coordinates with respect to the COM, and $\kappa$ the grid values at those points.

Since the integral over the unit circle
$
\begin{equation}
\frac{1}{\pi} \int x^2 \mathrm{d}x \mathrm{d}y = \frac{1}{\pi} \int r^{3} \mathrm{d}r \sin^{2}{\phi} \mathrm{d}\phi = \frac{1}{4}
\end{equation}
$

gives a factor of $1/4$. Therefore, the eigenvalues of the inertial tensor are $a^{2}/4$ and $b^{2}/4$, where $a$ and $b$ are the semi-major, resp. minor, axes.

In [ ]:
def inertial_tensor(kappa_map, pixel_scale=1, activation=None, round_com=True):
    com = center_of_mass(kappa_map, pixel_scale=pixel_scale)
    if not np.any(com):
        com = np.array([s//2 for s in kappa_map.shape])
    if round_com:
        com = np.round(com)
    if activation is not None:
        # kappa_map[kappa_map >= activation] = 1
        kappa_map[kappa_map < activation] = 0
    y, x = np.indices(kappa_map.shape)
    y = y*pixel_scale - com[1]
    x = x*pixel_scale - com[0]
    xx = x*x
    yy = y*y
    yx = xy = y*x
    N = 1./np.sum(kappa_map)
    Ixx = N*np.sum(kappa_map*xx)
    Ixy = N*np.sum(kappa_map*xy)
    Iyx = N*np.sum(kappa_map*yx)
    Iyy = N*np.sum(kappa_map*yy)
    return np.matrix([[Ixx, Ixy], [Iyx, Iyy]])


# on 17x17 grids
print(eagle_kappa_resmap.shape)
eagle_QPM = inertial_tensor(eagle_kappa_resmap, activation=1)
print(eagle_QPM)
# print(inertial_tensor(eagle_kappa_resmap, glass_cell_size))

print(min_e2gkappa_map.shape)
glass_QPM = inertial_tensor(min_e2gkappa_map, activation=1)
print(glass_QPM)
# print(inertial_tensor(min_e2gkappa_map, glass_cell_size))


# on 161x161 grids
#print(eagle_kappa_map.shape)
#eagle_QPM = inertial_tensor(eagle_kappa_map)
#print(eagle_QPM)
## print(inertial_tensor(eagle_kappa_resmap, eagle_cell_size))

#print(min_g2ekappa_map.shape)
#glass_QPM = inertial_tensor(min_g2ekappa_map)
#print(glass_QPM)
## print(inertial_tensor(min_g2ekappa_map, eagle_cell_size))



In [ ]:
def eigvals2x2(q):
    """
    Calculate the eigenvalues for a 2 by 2 matrix

    Args:
        q <np.matrix> - the matrix for which to calculate the eigenvalues

    Kwargs:
        None

    Return:
        lambda1, lambda2 <float,float> - the two eigenvalues
    """
    a, b, c, d = q.flatten().tolist()[0]
    T = a + d
    D = a*d-b*c
    lambda1 = 0.5*T + np.sqrt(0.25*T*T - D)
    lambda2 = 0.5*T - np.sqrt(0.25*T*T - D)
    return lambda1, lambda2


def diag2x2(q):
    """
    Calculate the diagonal matrix of a 2 by 2 matrix

    Args:
        q <np.matrix> - the matrix for which to calculate the eigenvalues

    Kwargs:
        None

    Return:
        d <np.matrix> - the diagonal matrix of q
    """
    l1, l2 = eigvals2x2(q)
    if l1 != l2:
        return np.matrix([[l1, 0], [0, l2]])
    else:
        None

def vangle(v1, v2):
    """
    Find the angle between two 2d vectors

    Args:
        v1, v2 <np.ndarray> - the vectors

    Kwargs:
        None

    Return:
        angle <float> - the angle between the vectors
    """
    v1 = np.asarray(v1)
    v2 = np.asarray(v2)
    vdotv = v1.dot(v2)
    vvnorm = np.sqrt(v1.dot(v1)) * np.sqrt(v2.dot(v2))
    return np.arccos(vdotv/(vvnorm))
    
def eigvec2x2(q):
    """
    Calculate the eigenvectors for a 2 by 2 matrix

    Args:
        q <np.matrix> - the matrix for which to calculate the eigenvalues

    Kwargs:
        None

    Return:
        v <np.ndarray, np.ndarray> - the eigenvectors of q
    """
    a, b, c, d = q.flatten().tolist()[0]
    ratio = np.array([(a+c-li)/(li-b-d) for li in eigvals2x2(q)])
    v1 = np.array([1, 1])  # choose something
    v = np.array([v1, ratio*v1]).T
    
    # assert vangle(v[0], v[1]) - 0.5*np.pi < 1e-12
    return v[0], v[1]


print("EAGLE model")
eagle_eigvals = eigvals2x2(eagle_QPM)
eagle_eigvecs = eigvec2x2(eagle_QPM)
eagle_phi = vangle(eagle_eigvecs[0], [1, 0])
eagle_a, eagle_b = [2*np.sqrt(l) for l in eagle_eigvals]

# print(diag2x2(eagle_QPM))
# print(eigvec2x2(eagle_QPM))
print(eagle_a, eagle_b)
print(eagle_phi * 180/np.pi)

print("GLASS model")
glass_eigvals = eigvals2x2(glass_QPM)
glass_eigvecs = eigvec2x2(glass_QPM)
glass_phi = vangle(glass_eigvecs[0], [1, 0])
glass_a, glass_b = [2*np.sqrt(l) for l in glass_eigvals]

# print(diag2x2(glass_QPM))
# print(eigvec2x2(glass_QPM))
print(glass_a, glass_b)
print(glass_phi * 180/np.pi)



## Loop over all 3 test cases (H3S0A0B90G0, H10S0A0B90G0, H36S0A0B90G0)

In [ ]:
%%skip True
for k in ["H3S0A0B90G0", "H10S0A0B90G0", "H36S0A0B90G0"]:
    filepath = kappa_map_files[k][0]
    eagle_model = fits.getdata(filepath, header=True)

    for idx in range(len(states[k])):
        statefile = states[k][idx]
        fltrdfile = filtered_states[k][idx]
        synthfile = fsynth50_states[k][idx]
        fltrdsynthfile = prefiltered_fsynth50_states[k][idx]

        for f in [statefile, fltrdfile, synthfile, fltrdsynthfile]:
            print(f)

            glass_state = glass.glcmds.loadstate(f)
            glass_state.make_ensemble_average()

            N_models = len(glass_state.models)
            maprad = glass_state.models[0]['obj,data'][0][0].basis.top_level_cell_size \
                     * glass_state.models[0]['obj,data'][0][0].basis.pixrad
            extent = (-maprad, maprad, -maprad, maprad)
            print("Number of models: {:4d}".format(N_models))
            print("Maprad: {:02.4f}".format(maprad))

            deltas = []
            for i, m in enumerate(glass_state.models):
                objdta = m['obj,data'][0]
                glass_kappa_map = objdta[0].basis._to_grid(objdta[1]['kappa'], 1)

                eagle_kappa_resmap = resample_eaglemodel(eagle_model, extent, glass_kappa_map.shape)

                delta_map = eagle_kappa_resmap - glass_kappa_map
                delta2_map = resid_map * resid_map
                deltas.append(np.sum(delta2_map))

            minidx = np.argmin(deltas)
            print("Closest model: Index {} with a delta^2 of {}".format(minidx, residuals[minidx]))
            objdta = glass_state.models[minidx]['obj,data'][0]
            glass_kappa_map = objdta[0].basis._to_grid(objdta[1]['kappa'], 1)
            eagle_kappa_resmap = resample_eaglemodel(eagle_model, extent, glass_kappa_map.shape)
            
    